In [30]:
# import gspread as gs
import pandas as pd
from IPython.display import Markdown, display
import plotly.express as px

In [23]:
# Get credentials for gspread
# gc = gs.service_account(filename="service_account.json")

# Open Google Sheets worksheets
# sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/12hULH8zk8njDnBph7zEuEtb8nSyKOOGqMTpU45iS8Vg/edit?usp=sharing")

url2 = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTkaDvWWgl_Qzk8HwuwW4UkZUt2kd443Nh6VQ9R0pEGoBZWZVAPAZDP9_b101E0fg6XPOB9A45WgAHq/pub?gid=2044430038&single=true&output=csv"
df1 = pd.read_csv(url2)

# ws1 = sh.worksheet("patientsstatus")
# df1 = pd.DataFrame(ws1.get_all_records())

# ws2 = sh.worksheet("personaldetails")
# df2 = pd.DataFrame(ws2.get_all_records())

# Format data into proper data types
df1["Timestamp"] = pd.to_datetime(df1["Timestamp"])
df1 = df1.sort_values(by="Timestamp", ascending=False)
df1

,Timestamp,Parsed_Date,ID,default,dead,Length_Height,Weight,MUAC,Edema,Zscore,...,isdead,isnormal,lheight,L,M,S,Brgy,Muni_city,Province,Region
1,2022-07-27 23:35:43.610000+00:00,7/27/2022 23:35:43,rqIFv4aa,no,no,65.0,5.7,14.0,yes,-3.39,...,0,0,65.0,-0.3521,7.4327,0.08217,EM's Barrio,Legazpi City,Albay,Region V
5,2022-07-25 13:35:43.610000+00:00,7/25/2022 13:35:43,aLgyCy,no,yes,85.0,12.0,14.0,no,0.52,...,1,0,85.0,-0.3521,11.5007,0.08181,EM's Barrio,Legazpi City,Albay,Region V
2,2022-07-24 13:35:43.610000+00:00,7/24/2022 13:35:43,vCYZlf,no,no,90.5,10.5,14.0,no,-2.59,...,0,0,90.5,-0.3521,12.8392,0.08034,EM's Barrio,Legazpi City,Albay,Region V
4,2022-07-23 13:35:43.610000+00:00,7/23/2022 13:35:43,jD7Z4U,yes,no,85.0,12.0,14.0,no,0.52,...,0,0,85.0,-0.3521,11.5007,0.08181,EM's Barrio,Legazpi City,Albay,Region V
3,2022-07-22 13:35:43.610000+00:00,7/22/2022 13:35:43,jAElxR,no,no,47.0,6.0,14.0,no,7.31,...,0,1,47.0,-0.3833,2.8007,0.09044,Cruzada,Legazpi City,Albay,Region V
0,2022-07-13 03:40:58.610000+00:00,7/13/2022 3:40:58,rqIFv4aa,no,no,65.0,5.7,14.0,no,-3.39,...,0,0,65.0,-0.3521,7.4327,0.08217,EM's Barrio,Legazpi City,Albay,Region V


In [24]:
# Sort value by most recent to least recent and keep only latest records for each unique child
df1_recent = df1.sort_values(by="Timestamp", ascending=False).drop_duplicates( subset=["ID"], keep="first")

# Summarize SAM, MAM, normal, defaulted cases by day then barangay
df1_recent = df1_recent.groupby([pd.Grouper(key='Timestamp', freq='D'), 'Brgy', 'Muni_city', 'Province','Region'])[['SAM','MAM','isdefault','isdead','isnormal']].sum().reset_index().sort_values('Timestamp')

# Store values of currently active SAM and MAM cases
current_active_SAM = df1_recent['SAM'].sum()
current_active_MAM = df1_recent['MAM'].sum()
recent_defaulted = df1_recent['isdefault'].sum()
recent_dead = df1_recent['isdead'].sum()

df1_recent

,Timestamp,Brgy,Muni_city,Province,Region,SAM,MAM,isdefault,isdead,isnormal
0,2022-07-22 00:00:00+00:00,Cruzada,Legazpi City,Albay,Region V,0,0,0,0,1
1,2022-07-23 00:00:00+00:00,EM's Barrio,Legazpi City,Albay,Region V,0,0,1,0,0
2,2022-07-24 00:00:00+00:00,EM's Barrio,Legazpi City,Albay,Region V,0,1,0,0,0
3,2022-07-25 00:00:00+00:00,EM's Barrio,Legazpi City,Albay,Region V,0,0,0,1,0
4,2022-07-27 00:00:00+00:00,EM's Barrio,Legazpi City,Albay,Region V,1,0,0,0,0


In [25]:
# Aggregate all SAM, MAM, default, dead, normal results per barangay
df1_grouped = df1.groupby([pd.Grouper(key='Timestamp', freq='D'), 'Brgy', 'Muni_city', 'Province','Region'])[['SAM','MAM','isdefault','isdead','isnormal']].sum().reset_index().sort_values('Timestamp')

total_SAM = df1_grouped['SAM'].sum()
total_MAM = df1_grouped['MAM'].sum()
total_default = df1_grouped['isdefault'].sum()
total_dead = df1_grouped['isdead'].sum()

df1_grouped

,Timestamp,Brgy,Muni_city,Province,Region,SAM,MAM,isdefault,isdead,isnormal
0,2022-07-13 00:00:00+00:00,EM's Barrio,Legazpi City,Albay,Region V,1,0,0,0,0
1,2022-07-22 00:00:00+00:00,Cruzada,Legazpi City,Albay,Region V,0,0,0,0,1
2,2022-07-23 00:00:00+00:00,EM's Barrio,Legazpi City,Albay,Region V,0,0,1,0,0
3,2022-07-24 00:00:00+00:00,EM's Barrio,Legazpi City,Albay,Region V,0,1,0,0,0
4,2022-07-25 00:00:00+00:00,EM's Barrio,Legazpi City,Albay,Region V,0,0,0,1,0
5,2022-07-27 00:00:00+00:00,EM's Barrio,Legazpi City,Albay,Region V,1,0,0,0,0


In [26]:
# Get all records of child with user_id of a1 for example
user = "rqIFv4aa"
df1_history = df1.loc[df1["ID"] == user][['Timestamp','default','dead','Length_Height','Weight','MUAC','Edema','Zscore','Age','Status']].sort_values('Timestamp', ascending=False)
df1_history

,Timestamp,default,dead,Length_Height,Weight,MUAC,Edema,Zscore,Age,Status
1,2022-07-27 23:35:43.610000+00:00,no,no,65.0,5.7,14.0,yes,-3.39,4,SEVERE Acute Malnutrition
0,2022-07-13 03:40:58.610000+00:00,no,no,65.0,5.7,14.0,no,-3.39,4,SEVERE Acute Malnutrition


In [27]:
display(Markdown(f'<table><tr><th><div><p style="font-weight:bold; font-size: 1.25em;">Active SAM cases</p><h4 style="font-size:3em;">{current_active_SAM}</h4></div></th><th><div><p style="font-weight:bold; font-size: 1.25em;">Active MAM cases</p><h4 style="font-size:3em;">{current_active_MAM}</h4></div></th><th><div><p style="font-weight:bold; font-size: 1.25em;">Recent Defaults</p><h4 style="font-size:3em;">{recent_defaulted}</h4></div></th><th><div><p style="font-weight:bold; font-size: 1.25em;">Recent Deaths</p><h4 style="font-size:3em;">{recent_dead}</h4></div></th></tr></table>'))

<table><tr><th><div><p style="font-weight:bold; font-size: 1.25em;">Active SAM cases</p><h4 style="font-size:3em;">1</h4></div></th><th><div><p style="font-weight:bold; font-size: 1.25em;">Active MAM cases</p><h4 style="font-size:3em;">1</h4></div></th><th><div><p style="font-weight:bold; font-size: 1.25em;">Recent Defaults</p><h4 style="font-size:3em;">1</h4></div></th><th><div><p style="font-weight:bold; font-size: 1.25em;">Recent Deaths</p><h4 style="font-size:3em;">1</h4></div></th></tr></table>

In [28]:
# display(Markdown(f'## Total SAM: {total_SAM}<br/>Total MAM: {total_MAM}'))
display(Markdown(f'<table><tr><th><div><p style="font-weight:bold; font-size: 1.25em;">Total SAM cases</p><h4 style="font-size:3em;">{total_SAM}</h4></div></th><th><div><p style="font-weight:bold; font-size: 1.25em;">Total MAM cases</p><h4 style="font-size:3em;">{total_MAM}</h4></div></th><th><div><p style="font-weight:bold; font-size: 1.25em;">Total Defaults</p><h4 style="font-size:3em;">{total_default}</h4></div></th><th><div><p style="font-weight:bold; font-size: 1.25em;">Total Deaths</p><h4 style="font-size:3em;">{total_dead}</h4></div></th></tr></table>'))


<table><tr><th><div><p style="font-weight:bold; font-size: 1.25em;">Total SAM cases</p><h4 style="font-size:3em;">2</h4></div></th><th><div><p style="font-weight:bold; font-size: 1.25em;">Total MAM cases</p><h4 style="font-size:3em;">1</h4></div></th><th><div><p style="font-weight:bold; font-size: 1.25em;">Total Defaults</p><h4 style="font-size:3em;">1</h4></div></th><th><div><p style="font-weight:bold; font-size: 1.25em;">Total Deaths</p><h4 style="font-size:3em;">1</h4></div></th></tr></table>

In [82]:
# If you prefer to run the code online instead of on your computer click:
# https://github.com/Coding-with-Adam/Dash-by-Plotly#execute-code-in-browser

from dash import Dash, dcc, html, Output, Input  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
import plotly.express as px

# incorporate data into app
df = px.data.medals_long()

# Build your components
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
mytitle = dcc.Markdown(children='# Acute Malnutrition Case Tracker')

left_jumbotron1 = dbc.Col(
    html.Div(
        [
            html.P("Active SAM cases", className="font-weight-bold", style={'font-size': '1em'}),
            html.H4(current_active_SAM, className="display-6",),
        ],
        className="h-100 p-5 text-white bg-danger rounded-3",
    ),
    md=3,
)

middlel_jumbotron1 = dbc.Col(
    html.Div(
        [
            html.P("Active MAM cases", className="font-weight-bold", style={'font-size': '1em'}),
            html.H4(current_active_MAM, className="display-6",),
        ],
        className="h-100 p-5 text-white bg-warning rounded-3",
    ),
    md=3,
)

middler_jumbotron1 = dbc.Col(
    html.Div(
        [
            html.P("Recent Defaults", className="font-weight-bold", style={'font-size': '1em'}),
            html.H4(recent_defaulted, className="display-6",),
        ],
        className="h-100 p-5 text-white bg-dark rounded-3",
    ),
    md=3,
)

right_jumbotron1 = dbc.Col(
    html.Div(
        [
            html.P("Recent Deaths", className="font-weight-bold", style={'font-size': '1em'}),
            html.H4(recent_dead, className="display-6",),
        ],
        className="h-100 p-5 text-white bg-dark rounded-3",
    ),
    md=3,
)

jumbotron1 = dbc.Row(
    [left_jumbotron1, middlel_jumbotron1, middler_jumbotron1, right_jumbotron1],
    className="align-items-md-stretch",
    style={'margin': '1em'}
)

left_jumbotron2 = dbc.Col(
    html.Div(
        [
            html.P("Total SAM", className="font-weight-bold", style={'font-size': '1em'}),
            html.H4(total_SAM, className="display-6",),
        ],
        className="h-100 p-5 text-white bg-danger rounded-3",
    ),
    md=3,
)

middlel_jumbotron2 = dbc.Col(
    html.Div(
        [
            html.P("Total MAM", className="font-weight-bold", style={'font-size': '1em'}),
            html.H4(total_MAM, className="display-6",),
        ],
        className="h-100 p-5 text-white bg-warning rounded-3",
    ),
    md=3,
)

middler_jumbotron2 = dbc.Col(
    html.Div(
        [
            html.P("Total Default", className="font-weight-bold", style={'font-size': '1em'}),
            html.H4(total_default, className="display-6",),
        ],
        className="h-100 p-5 text-white bg-dark rounded-3",
    ),
    md=3,
)

right_jumbotron2 = dbc.Col(
    html.Div(
        [
            html.P("Total Deaths", className="font-weight-bold", style={'font-size': '1em'}),
            html.H4(total_dead, className="display-6",),
        ],
        className="h-100 p-5 text-white bg-dark rounded-3",
    ),
    md=3,
)

jumbotron2 = dbc.Row(
    [left_jumbotron2, middlel_jumbotron2, middler_jumbotron2, right_jumbotron2],
    className="align-items-md-stretch",
    style={'margin': '1em'}
)

jumbotrongraph = html.Div(
    dbc.Container(
        [
            html.H5("Case Data by Date Classified", className="display-5"),
            html.P(
                "This graph shows the number of cases of each type of case by date."
            ),
            html.Hr(className="my-2"),
        ],
        fluid=True,
        className="py-3",
    ),
    className="p-3 bg-light rounded-3",
)

mygraph = dcc.Graph(figure={})

# jumbomode = dbc.Col(
#     html.Div(
#         [
#             html.P("Data to be Shown: ", className="font-weight-bold", style={'font-size': '1em'}),
#             dcc.Dropdown(options=['SAM', 'MAM', 'Defaults', 'Deaths'],
#                         value='SAM',  # initial value displayed when page first loads
#                         clearable=False),
#             # dbc.DropdownMenu([dbc.DropdownMenuItem('SAM'), dbc.DropdownMenuItem('MAM'), dbc.DropdownMenuItem('Defaults'), dbc.DropdownMenuItem('Deaths')],
#             #     label="Data", color="primary", className="m-1"
#             # ),
        
#             html.P("Date Mode: ", className="font-weight-bold", style={'font-size': '1em'}),
#             dcc.Dropdown(options=['Daily', 'Weekly'],
#                         value='Daily',  # initial value displayed when page first loads
#                         clearable=False)
#             # dbc.DropdownMenu([dbc.DropdownMenuItem('Daily'), dbc.DropdownMenuItem('Weekly')],
#             #     label="Frequency", color="primary", className="m-1"
#             # ),
#         ],
#         className="h-100 p-5 text-black bg-light rounded-3",
#     ),
#     md=12,
# )

# jumbotron3 = dbc.Row(
#     [jumbomode],
#     className="align-items-md-stretch",
#     style={'margin': '1em'}
# )

dropdown_mode = dcc.Dropdown(options=['SAM', 'MAM', 'Defaults', 'Deaths'],
                        value='SAM',  # initial value displayed when page first loads
                        clearable=False)
dropdown_datemode = dcc.Dropdown(options=['Daily', 'Weekly'],
                        value='Daily',  # initial value displayed when page first loads
                        clearable=False)


# Customize your own Layout
app.layout = dbc.Container([mytitle, jumbotron1, jumbotron2, jumbotrongraph, dropdown_mode, dropdown_datemode, mygraph])

# Callback allows components to interact
@app.callback(
    Output(mygraph, component_property='figure'),
    Input(dropdown_mode, component_property='value'),
    Input(dropdown_datemode, component_property='value')
)

def update_graph(mode, datemode):  # function arguments come from the component property of the Input
    if mode == "SAM":
        data_mode = "SAM"
        bary_label = "SAM cases"
    elif mode == "MAM":
        data_mode = "MAM"
        bary_label = "MAM cases"
    elif mode == "Defaults":
        data_mode = "isdefault"
        bary_label = "Defaulted cases"
    elif mode == "Deaths":
        data_mode = "isdead"
        bary_label = "Deaths"

    if datemode == "Daily":
        bar_freq = "D"
    elif datemode == "Weekly":
        bar_freq = "W"

    df1_grouped = df1.groupby([pd.Grouper(key='Timestamp', freq=bar_freq), 'Brgy', 'Muni_city', 'Province','Region'])[['SAM','MAM','isdefault','isdead','isnormal']].sum().reset_index().sort_values('Timestamp')

    fig = px.bar(df1_grouped, x="Timestamp", y=data_mode, labels={
                     "Timestamp": "Date",
                     data_mode: bary_label,
                 })

    return fig  # returned objects are assigned to the component property of the Output


def update_graph(mode, datemode):  # function arguments come from the component property of the Input
    if mode == "SAM":
        data_mode = "SAM"
        bary_label = "SAM cases"
    elif mode == "MAM":
        data_mode = "MAM"
        bary_label = "MAM cases"
    elif mode == "Defaults":
        data_mode = "isdefault"
        bary_label = "Defaulted cases"
    elif mode == "Deaths":
        data_mode = "isdead"
        bary_label = "Deaths"

    if datemode == "Daily":
        bar_freq = "D"
    elif datemode == "Weekly":
        bar_freq = "W"

    df1_grouped = df1.groupby([pd.Grouper(key='Timestamp', freq=bar_freq), 'Brgy', 'Muni_city', 'Province','Region'])[['SAM','MAM','isdefault','isdead','isnormal']].sum().reset_index().sort_values('Timestamp')

    fig = px.bar(df1_grouped, x="Timestamp", y=data_mode, labels={
                     "Timestamp": "Date",
                     data_mode: bary_label,
                 })

    return fig  # returned objects are assigned to the component property of the Output


# Run app
if __name__=='__main__':
    app.run_server(port=8080)

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is running on http://127.0.0.1:8080/

Dash is run

 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jul/2022 09:45:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2022 09:45:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2022 09:45:36] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2022 09:45:36] "GET /_favicon.ico?v=2.6.0 HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2022 09:45:37] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2022 09:45:37] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2022 09:45:37] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2022 09:45:37] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2022 09:45:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2022 09:45:37] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/20

In [54]:
mode = 'SAM'
if mode == "SAM":
    data_mode = "SAM"
    bary_label = "SAM cases"
elif mode == "MAM":
    data_mode = "SAM"
    bary_label = "MAM cases"
elif mode == "Defaults":
    data_mode = "isdefault"
    bary_label = "Defaulted cases"
elif mode == "Deaths":
    data_mode = "isdead"
    bary_label = "Deaths"

date_mode = "Daily"
if date_mode == "Daily":
    bar_freq = "D"
elif date_mode == "Weekly":
    bar_freq = "W"

df1_grouped = df1.groupby([pd.Grouper(key='Timestamp', freq=bar_freq), 'Brgy', 'Muni_city', 'Province','Region'])[['SAM','MAM','isdefault','isdead','isnormal']].sum().reset_index().sort_values('Timestamp')

fig = px.bar(df1_grouped, x="Timestamp", y=data_mode, labels={
                     "Timestamp": "Date",
                     data_mode: bary_label,
                 })
fig.show()

In [ ]:
# Get all records of child with user_id of a1 for example
user = "rqIFv4aa"
df1_history = df1.loc[df1["ID"] == user][['Timestamp','default','dead','Length_Height','Weight','MUAC','Edema','Zscore','Age','Status']].sort_values('Timestamp', ascending=False)
df1_history

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c10d3b81-c7bb-40bf-98a8-a9afcc527b5c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>